In [13]:
import os,sys
import pandas as pd
import numpy as np
from time import sleep, time
from datetime import date
import threading
from typing import Tuple
from datasets import load_dataset
from openai import OpenAI
import google.generativeai as genai
from google.ai import generativelanguage as glm
import json
from datetime import datetime
import ast


### Import human dataset

In [14]:
# Load the data
original_human_dataset = pd.read_csv('../generated_dataset/complete_dataset.csv')
len(original_human_dataset)

760

### ```human_dataset``` dataframe creation

In [15]:
human_dataset = original_human_dataset.copy()
print("Number of rows in human dataset: ", len(human_dataset))
unique_video_ids_human = human_dataset['video_id'].nunique()
print("Number of videos: ", unique_video_ids_human)

Number of rows in human dataset:  760
Number of videos:  288


### Construct ```df_transcripts``` from ```human_dataset```
this has 
- unique `video_id`
- `video_title`
- `transcript`

In [16]:
# Assuming `human_dataset` is your initial DataFrame
# Dropping duplicate rows based on `video_id` and keeping the first occurrence
unique_videos = human_dataset.drop_duplicates(subset='video_id', keep='first')

# Extracting the final DataFrame with the required columns
df_transcripts = unique_videos[['video_id', 'video_title', 'transcript']]
df_transcripts


,video_id,video_title,transcript
0,0CJU8R4oNFk,5 Stocks to Buy Now to Double Your Money,"Hey Bowtie Nation, Joseph Hogue here with the..."
5,0Fg0YsbOzJA,i am selling it,So we have decided to start selling out of Wy...
6,0OJIHD_o59M,The Best Internet Stocks for 2023 You Can Buy Now,"Hey Bowtie Nation, Joseph Hogue here and a ve..."
9,1Gm4A7EFYI4,I Just Bought The PERFECT Dividend Stock (At A...,So I've had my eye on a few different stocks ...
10,1Lx7z_x4Rc0,🔵WARNING TO EVERYONE!!!🔵 I JUST SOLD IT ALL!!!,Family we absolutely dominated it today. If y...
...,...,...,...
755,Y61LJ2Pwsgw,I’m Buying This NEW Growth Stock ETF! 💰 $DKNG ...,"Hey guys, Eric here, Mr. Fired Up Wealth. I w..."
756,Y8iGpTUtrvI,7 Best ETFs to Buy to Grow Your Money,"Hey Bowtie Nation, Joseph Hogue here and grow..."
757,zxji3hEMdOM,Should You Buy or Rent a Home in 2024? | The A...,The average cost to rent over a lifetime is w...
758,_KdRpU1CMzY,URGENT!⛔️ SHIBA INU PRICE PREDICTION IS POPPIN...,Crypto is exploding up. We are seeing a few t...


### Check: unique ```video_id```s for human_dataset and df_transcripts
##### This is to make sure that we are using the same videos for inference (`df_transcripts`) and the same videos for comparison/evaluation (`human_dataset`)

In [17]:
# Calculate unique video IDs in both datasets
unique_video_ids_human = human_dataset['video_id'].nunique()
unique_video_ids_transcripts = df_transcripts['video_id'].nunique()

# Check if the unique video IDs are the same
unique_video_ids_human_set = set(human_dataset['video_id'].unique())
unique_video_ids_transcripts_set = set(df_transcripts['video_id'].unique())

# Compare the sets
are_same = unique_video_ids_human_set == unique_video_ids_transcripts_set

# Print the results
print(f"Unique video IDs in human dataset: {unique_video_ids_human}")
print(f"Unique video IDs in df_transcripts: {unique_video_ids_transcripts}")
print(f"Are the unique video IDs the same? {are_same}")

Unique video IDs in human dataset: 288
Unique video IDs in df_transcripts: 288
Are the unique video IDs the same? True


In [18]:
df_transcripts.to_csv('../generated_dataset/w_transcripts_for_inference.csv', index=False)